In [2]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
from torch.utils.data import Subset, ChainDataset, DataLoader, random_split
import torch.nn.functional as F

import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True

import pandas as pd
import pickle
import math
import random

import HSIC
import matplotlib.pyplot as plt
import numpy as np
import func
import torchy

import wandb

from datetime import datetime
from torch.utils.tensorboard import SummaryWriter


In [3]:
a_file = open("PACS.pkl", "rb")
full_dict = pickle.load(a_file)


In [4]:
domain_names = ['photo', 'art_painting', 'cartoon']

data_list, labels = func.dict_to_data(full_dict, domain_names)

In [5]:
all_domain_names = ['photo', 'art_painting', 'cartoon', 'sketch']
domain_names = ['photo', 'art_painting', 'cartoon']

# data_dict = {}
# for name in domain_names:
#     data_dict[name] = full_dict[name]
    
num_classes = 7      # 7 classes for each domain: 'dog', 'elephant', 'giraffe', 'guitar', 'horse', 'house', 'person'
classes_names = ['Dog', 'Elephant', 'Giraffe', 'Guitar', 'Horse', 'House', 'Person']
domain_mapping = {'photo':0, 'art_painting':1, 'cartoon':2, 'sketch':3}
model_path = 'saved_model.pth'

criterion = nn.CrossEntropyLoss()


In [ ]:
lambd = 3
i = 0
net = torchy.Net()
lr=0.001
use_hsic = True
batch_size = 128




for lambd in [0, 1, 2, 3, 5, 8]:
    print('Lambda: ', lambd)
    i=0
    for name in domain_names:
        for name2 in domain_names:
            if name == name2: 
                continue
            train_names = [name, name2]
            valid_name = [n for n in domain_names if n not in train_names]
            test_name = 'split'

            train_loader, valid_loader, test_loader  = torchy.get_feature_loaders(full_dict, 
                                                                                   batch_size,
                                                                                   train_names,
                                                                                   valid_name,
                                                                                   test_name,
                                                                                 verbose=False)


            net = torchy.Net()

            min_valid_loss = 1000

            writer= SummaryWriter(f'runs/feature/lambda{lambd}/{i}/{use_hsic}')

            optimizer = optim.Adam(net.parameters(), lr=lr)
            min_valid_loss = func.train(net, criterion, optimizer, 
                                        train_loader,
                                        valid_loader=valid_loader,
                                        epochs=30,
                                        use_hsic=use_hsic,
                                        lambd=lambd,
                                        writer=writer,
                                        min_valid_loss = min_valid_loss)
            writer.close()

            acc = func.test_model(test_loader, 'saved_model.pth')
            # acc_list.append(acc)
            print(f'{i} Trained: {train_names}, Valid: {valid_name}, Testing: {test_name}, HSIC: {use_hsic}= {acc}%')
            
            i+=1


In [8]:
net = torchy.Net()
lr=0.001
use_hsic = True
batch_size = 128
epochs = 30
lambd=5


wandb.init(project="test_one", entity="skohnie")


train_names = ['photo', 'cartoon']
valid_name = ['art_painting']
test_name = 'split'

# results =  {0:[], 5:[]}
# for lambd in [0, 5]:
#     print('Lambda: ', lambd)
#     for i in range(10):
train_loader, valid_loader, test_loader  = torchy.get_feature_loaders(full_dict, 
                                                                       batch_size,
                                                                       train_names,
                                                                       valid_name,
                                                                       test_name,
                                                                       verbose=False)


net = torchy.Net()

min_valid_loss = 1000

# writer= SummaryWriter(f'runs/test_one/lambda{lambd}/{i}/{use_hsic}')
wandb.config = {
    "learning_rate": lr,
    "epochs": 100,
    "batch_size": 128,
    "Lamda": lambd,
    "Train names": train_names,
    "Valid name": valid_name,
    "Test name": test_name
}


optimizer = optim.Adam(net.parameters(), lr=lr)
min_valid_loss = func.train(net, criterion, optimizer, 
                            train_loader,
                            valid_loader=valid_loader,
                            epochs=30,
                            use_hsic=use_hsic,
                            lambd=lambd,
                            writer=None,
                            min_valid_loss = min_valid_loss,
                            wb=True)

acc = func.test_model(test_loader, 'saved_model.pth')
print(acc)
#         results[lambd].append(acc)
        # acc_list.append(acc)
# print(f'{i} Trained: {train_names}, Valid: {valid_name}, Testing: {test_name}, HSIC: {use_hsic}= {acc}%')




wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ········


wandb: ERROR API key must be 40 characters long, yours was 1859
wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ········


Traceback (most recent call last):
  File "/opt/anaconda3/envs/thesis/lib/python3.9/site-packages/wandb/sdk/wandb_init.py", line 993, in init
    wi.setup(kwargs)
  File "/opt/anaconda3/envs/thesis/lib/python3.9/site-packages/wandb/sdk/wandb_init.py", line 241, in setup
    wandb_login._login(
  File "/opt/anaconda3/envs/thesis/lib/python3.9/site-packages/wandb/sdk/wandb_login.py", line 284, in _login
    wlogin.prompt_api_key()
  File "/opt/anaconda3/envs/thesis/lib/python3.9/site-packages/wandb/sdk/wandb_login.py", line 212, in prompt_api_key
    key, status = self._prompt_api_key()
  File "/opt/anaconda3/envs/thesis/lib/python3.9/site-packages/wandb/sdk/wandb_login.py", line 192, in _prompt_api_key
    key = apikey.prompt_api_key(
  File "/opt/anaconda3/envs/thesis/lib/python3.9/site-packages/wandb/sdk/lib/apikey.py", line 123, in prompt_api_key
    write_key(settings, key, api=api)
  File "/opt/anaconda3/envs/thesis/lib/python3.9/site-packages/wandb/sdk/lib/apikey.py", line 217, in

Exception: problem

In [ ]:
X = np.array((results[0], results[5]))
print(X)
print(X.mean(axis=1))
print(X.std(axis=1))


In [ ]:
lambd = 0
i = 0

train_names = [DATASETS_NAMES[0]]
valid_name = [DATASETS_NAMES[1]]
test_name = [DATASETS_NAMES[3]]
HSIC_name = [DATASETS_NAMES[2]]
train_loader, valid_loader, test_loader, HSIC_loader  = torchy.get_loaders(feature_dict, [DATASETS_NAMES[0]], DATASETS_NAMES, BATCH_SIZE, val_test_domains=True, 
                                                                                    with_hsic=with_hsic, valid_name=[DATASETS_NAMES[1]], test_name=[DATASETS_NAMES[2]])
acc_list = []                          
min_valid_loss = 1000
for lr in [0.02, 0.001]:
    net = torchy.Net()
    writer= SummaryWriter(f'runs/test_bn/l:{lambd}/i:{i}/lr:{lr}')
    i+=1
    print(i)
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)
    min_valid_loss = func.trainHSIC(net, criterion, optimizer, train_loader,
       validate_loader=valid_loader, epochs=20, HSIC_loader=HSIC_loader, lambd=lambd, writer=writer, min_valid_loss = min_valid_loss)
    writer.close()

acc = func.test_model(test_loader, 'saved_model.pth')
acc_list.append(acc)
print(f'Training: {train_names[0], HSIC_name[0] }, Valid: {valid_name[0]}, Testing: {test_name[0]} = {acc}%')

In [ ]:
PATH = 'saved_model.pth'
test_net = torchy.Net()
test_net.load_state_dict(torch.load(PATH))

test_net.eval()



train_iter = iter(train_loader)
HSIC_iter = iter(HSIC_loader)

data, labels1, _ = train_iter.next()

output = test_net(data)

def normalize_output(img):
    img = img - img.min()
    img = img / img.max()
    return img

# Plot some images
idx = torch.randint(0, output.size(0), ())
pred = normalize_output(output)
img = normalize_output(data)

fig, axarr = plt.subplots(1, 2)
print(img.shape)
axarr[0].imshow(img.detach().numpy())
axarr[1].imshow(pred.detach().numpy())

# Visualize feature maps
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook
print(test_net)

test_net.fc2.register_forward_hook(get_activation('fc1'))
data, labels1, _ = train_iter.next()
data = data
output = test_net(data)

act = activation['fc1'].squeeze()
fig, axarr = plt.subplots(1)
axarr.imshow(act)

data, labels1, _ = HSIC_iter.next()
data = data
output = test_net(data)

act = activation['fc1'].squeeze()
fig, axarr = plt.subplots(1)
axarr.imshow(act)

In [ ]:
net.fc1.register_forward_hook(get_activation('fc1'))
data, labels1, _ = train_iter.next()
data = data
output = net(data)

act = activation['fc1'].squeeze()
fig, axarr = plt.subplots(1)
axarr.imshow(act)

data, labels1, _ = HSIC_iter.next()
data = data
output = net(data)

act = activation['fc1'].squeeze()
fig, axarr = plt.subplots(1)
axarr.imshow(act)

In [ ]:
#!tensorboard --logdir=runs

In [ ]:
for l in [result_list1, result_list2, result_list3]:
    plt.hist(l, label= ['0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.9'], bins=5)
    plt.legend()
    plt.xlabel("Accuracy", fontsize=16)  
    plt.xticks(fontsize=14)  
    plt.yticks(fontsize=14)
    plt.show()

In [ ]:
# 1-hsic
result_list = [[65, 49, 74, 49, 74, 94, 69, 54, 94, 54, 94, 96, 61, 55, 93, 55, 93, 100, 66, 70, 93, 70, 93, 94], [68, 53, 74, 53, 74, 90, 67, 58, 94, 58, 94, 93, 61, 57, 94, 57, 94, 99, 64, 68, 93, 68, 93, 99], [65, 56, 75, 56, 75, 89, 69, 59, 94, 59, 94, 97, 66, 56, 93, 56, 93, 99, 63, 65, 95, 65, 95, 95], [63, 52, 75, 52, 75, 94, 72, 50, 91, 50, 91, 93, 65, 57, 95, 57, 95, 90, 65, 70, 93, 70, 93, 92], [61, 50, 73, 50, 73, 87, 68, 52, 93, 52, 93, 93, 63, 53, 92, 53, 92, 85, 65, 70, 94, 70, 94, 90]]

plt.hist(result_list, label= [str(i) for i in [0, 0.5, 2, 8, 11]], bins=4)
plt.legend()
plt.xlabel("Accuracy", fontsize=16)  
plt.xticks(fontsize=14)  
plt.yticks(fontsize=14)
plt.show()

In [ ]:
# writer=None
l=1
for lr in [0.5, 0.1, 0.05, 0.02, 0.01]:
    for l in [0]:
        net = torchy.Net()
        print(f'lr:{lr}_lam:{l}')
        writer= SummaryWriter(f'runs/test/lr:{lr}_lam:{l}')
        optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)
        func.trainHSIC(net, criterion, optimizer, train_loader,
           validate_loader=valid_loader, epochs=20, lambd=l, with_hsic=True, writer=writer)
        writer.close()

In [ ]:
for t in [True, False]:
    for lr in [0.05, 0.01, 0.005, 0.001]:
        if t == True:
            for l in [0, 0.1, 0.4, 0.9, 2]:
                net = torchy.Net()
                print(f'runs/{t}_lr:{lr}_lam:{l}')
                writer= SummaryWriter(f'runs/pa_s/{t}_lr:{lr}_lam:{l}')
                optimizer = optim.Adam(net.parameters(), lr=lr)
                func.trainHSIC(net, criterion, optimizer,  train_loader,
           validate_loader=valid_loader, epochs=20, lambd=l, with_hsic=t, writer=writer)
                writer.close()
        else:
            net = torchy.Net()
            print(f'runs/{t}_lr:{lr}')
            writer= SummaryWriter(f'runs/pa_s/{t}_lr:{lr}')
            optimizer = optim.Adam(net.parameters(), lr=lr)
            func.trainHSIC(net, criterion, optimizer,  train_loader,
           validate_loader=valid_loader, epochs=20, lambd=l, with_hsic=t, writer=writer)
            writer.close()  

In [ ]:
net = torchy.Net()

writer= SummaryWriter(f'runs/pa_s/{t}_lr:{lr}')
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)
func.trainHSIC(net, criterion, optimizer,  train_loader,
validate_loader=valid_loader, epochs=20, lambd=l, with_hsic=t, writer=writer)
writer.close()  

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs


# test_loader = cartoon_dataloader
test_loader = data_loaders['sketch']
# test_loader = cartoon_dataloader
PATH = 'saved_model.pth'
test_net = torchy.Net()
test_net.load_state_dict(torch.load(PATH))



with torch.no_grad():
    for data in test_loader:
        features, labels = data
        # calculate outputs by running images through the network
        outputs = test_net(features)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the {len(test_loader)} test images: {100 * correct // total} %')

In [ ]:



if True: print('bla')
